# Analyzing top hits playlists audio features by country 

## We will do:
- Analytics by country
- classifying tracks based on their audio features where the target variable is the country 

In [1]:
import MySQLdb
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sys
sys.path.append('../')
import myenvvar

In [2]:
conn = MySQLdb.Connection(
        host=myenvvar.db_vars['host'],
        user=myenvvar.db_vars['user'],
        passwd=myenvvar.db_vars['password'],
        port=myenvvar.db_vars['port'],
        db=myenvvar.db_vars['db']
        )
conn.set_character_set('utf8')

# Read top hits by country playlists and tracks

In [10]:
q = "select p.playlist_name, pt.playlist_id, af.* \
    from playlists p join playlists_tracks pt join audio_features af \
    on p.playlist_id = pt.playlist_id and pt.track_id = af.track_id \
    where p.playlist_name like 'Top Hits%'"
top_hits_by_country_df = pd.read_sql(q, conn)
top_hits_by_country_df.shape

(457, 19)

In [11]:
top_hits_by_country_df.head()

,playlist_name,playlist_id,track_id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,total_available_markets,popularity,added_datetime
0,Top Hits Greece,37i9dQZF1DX0e5BFIQWSDG,017PF4Q3l4DBUiWoXk4OWT,0.730,0.729,4,-3.434,0,0.0886,0.16700,0.00000,0.3490,0.467,113.012,221820,4,76,90,2020-06-12 08:59:52
1,Top Hits Greece,37i9dQZF1DX0e5BFIQWSDG,01Y0KGT7VzixtLyOj44x56,0.881,0.783,8,-3.070,0,0.1630,0.38200,0.00010,0.1010,0.545,102.040,155293,4,79,63,2020-06-12 08:59:52
2,Top Hits Greece,37i9dQZF1DX0e5BFIQWSDG,0dD17UWACV5aYTyeS6af1k,0.747,0.586,6,-8.799,1,0.0591,0.26100,0.00003,0.1100,0.176,99.986,194400,4,79,51,2020-06-12 08:59:52
3,Top Hits Greece,37i9dQZF1DX0e5BFIQWSDG,0nbXyq5TXYPCO7pr3N8S4I,0.896,0.586,10,-6.687,0,0.0559,0.10400,0.00000,0.7900,0.642,116.971,196653,4,79,96,2020-06-12 08:46:52
4,Top Hits Greece,37i9dQZF1DX0e5BFIQWSDG,0VjIjW4GlUZAMYd2vXMi3b,0.514,0.730,1,-5.934,1,0.0598,0.00146,0.00010,0.0897,0.334,171.005,200040,4,79,100,2020-06-12 08:49:52


# Get playlists names

In [12]:
playlists_names_list = np.unique(top_hits_by_country_df['playlist_name'])
playlists_names_list

array(['Top Hits Deutschland', 'Top Hits Greece', 'Top Hits Indonesia',
       'Top Hits Japan', 'Top Hits Philippines', 'Top Hits Polska',
       'Top Hits Portugal', 'Top Hits Switzerland', 'Top Hits Vietnam'],
      dtype=object)